# Week 14: Birth-and-Death Processes

**Objective:** Understand birth-and-death processes as a general framework for continuous-time Markov chains, derive the balance equations, and solve for the stationary distribution.

## Step 1: Build Intuition

Imagine a population (of people, customers in a store, molecules in a chemical reaction, etc.). Over time, the size of this population changes. A **birth-and-death process** is a simple yet powerful model for such a system, with one key restriction: the population size only ever changes by one at a time.

- A **"birth"** is an event that increases the population size by one (e.g., a customer arrives).
- A **"death"** is an event that decreases the population size by one (e.g., a customer finishes service and leaves).

Many processes we've already studied are actually birth-and-death processes! The M/M/1 queue is a perfect example: the state (number of customers) only increases by one (an arrival) or decreases by one (a departure).

## Step 2: Understand the Core Idea

A birth-and-death process is a special type of continuous-time Markov chain where transitions from state \(n\) are only allowed to state \(n+1\) or \(n-1\).

The entire process is defined by two sets of parameters:
1.  **Birth Rates (\(\lambda_n\)):** The rate at which births occur when the population size is \(n\). This is the rate of transition from state \(n\) to \(n+1\).
2.  **Death Rates (\(\mu_n\)):** The rate at which deaths occur when the population size is \(n\). This is the rate of transition from state \(n\) to \(n-1\).

Crucially, these rates can **depend on the current state \(n\)**. For example, in a queue with multiple servers, the death (service) rate depends on how many servers are busy. In a population, the birth rate might decrease if the population becomes overcrowded.

## Step 3: Learn the Definitions and Formulas

**Definition: Birth-and-Death Process**
A continuous-time Markov chain on the states \({0, 1, 2, ...}\) is a birth-and-death process if its transition rates \(q_{ij}\) are non-zero only for \(j = i+1\) or \(j = i-1\).
- \(q_{n, n+1} = \lambda_n\) (birth rate at state n)
- \(q_{n, n-1} = \mu_n\) (death rate at state n)

--- 

**Stationary Distribution (\(\pi_n\))**
We are often interested in the long-run probability of being in each state, \(\pi_n = P(X=n)\). To find this, we use the **balance equations**. The core idea is that in equilibrium, the rate of flow *out* of a state must equal the rate of flow *into* that state.

For any state \(n\), the detailed balance equation is:
$$ \text{Rate of leaving } n \text{ for } n+1 = \text{Rate of entering } n \text{ from } n+1 $$
$$ \pi_n \lambda_n = \pi_{n+1} \mu_{n+1} $$

We can solve this recursively:
\(\pi_1 = \frac{\lambda_0}{\mu_1} \pi_0\)
\(\pi_2 = \frac{\lambda_1}{\mu_2} \pi_1 = \frac{\lambda_1 \lambda_0}{\mu_2 \mu_1} \pi_0\)

This leads to the general solution for \(\pi_n\) in terms of \(\pi_0\):
$$ \pi_n = \pi_0 \prod_{i=0}^{n-1} \frac{\lambda_i}{\mu_{i+1}} $$

To find \(\pi_0\), we use the fact that all probabilities must sum to 1:
$$ \sum_{n=0}^{\infty} \pi_n = 1 \implies \pi_0 \left(1 + \sum_{n=1}^{\infty} \prod_{i=0}^{n-1} \frac{\lambda_i}{\mu_{i+1}}\right) = 1 $$
$$ \pi_0 = \frac{1}{1 + \sum_{n=1}^{\infty} \prod_{i=0}^{n-1} \frac{\lambda_i}{\mu_{i+1}}} $$

## Step 4: Apply and Practice

Let's show that this general framework gives us the exact same results for the **M/M/1 queue** we analyzed in Week 12.

**Scenario:** An M/M/1 queue with arrival rate \(\lambda\) and service rate \(\mu\).

**Mapping to a Birth-and-Death Process:**
- The state \(n\) is the number of customers in the system.
- **Births:** Arrivals happen at a constant rate, regardless of the number of customers. So, \(\lambda_n = \lambda\) for all \(n = 0, 1, 2, ...\).
- **Deaths:** Departures (service completions) happen only if there is a customer to serve (\(n > 0\)). The rate is constant as long as the server is busy. So, \(\mu_n = \mu\) for all \(n = 1, 2, ...\). (And \(\mu_0 = 0\)).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

### Part A: Deriving the M/M/1 Stationary Distribution

Let's use the general formula for \(\pi_n\) with our M/M/1 rates.
$$ \pi_n = \pi_0 \prod_{i=0}^{n-1} \frac{\lambda_i}{\mu_{i+1}} = \pi_0 \frac{\lambda_0}{\mu_1} \frac{\lambda_1}{\mu_2} \cdots \frac{\lambda_{n-1}}{\mu_n} $$
Substitute \(\lambda_i = \lambda\) and \(\mu_i = \mu\):
$$ \pi_n = \pi_0 \frac{\lambda}{\mu} \frac{\lambda}{\mu} \cdots \frac{\lambda}{\mu} \quad (n \text{ times}) $$
$$ \pi_n = \pi_0 \left(\frac{\lambda}{\mu}\right)^n = \pi_0 \rho^n $$
This is the same relationship we saw before! Now let's find \(\pi_0\) using the normalization condition:
$$ \sum_{n=0}^{\infty} \pi_n = \sum_{n=0}^{\infty} \pi_0 \rho^n = \pi_0 \sum_{n=0}^{\infty} \rho^n = 1 $$
The sum is a geometric series, which converges to \(\frac{1}{1-\rho}\) for \(\rho < 1\).
$$ \pi_0 \left(\frac{1}{1-\rho}\right) = 1 \implies \pi_0 = 1 - \rho $$
Therefore, the final stationary distribution is:
$$ \pi_n = (1 - \rho) \rho^n $$
This perfectly matches the known result for an M/M/1 queue, showing the power and generality of the birth-and-death framework.

### Part B: Visualizing the M/M/1 Distribution

Let's write a simple Python function to calculate and plot these probabilities.

In [ ]:
def plot_mm1_distribution(lambda_rate, mu_rate, max_n=20):
    """Calculates and plots the stationary distribution of an M/M/1 queue."""
    if lambda_rate >= mu_rate:
        print("Queue is unstable. No stationary distribution.")
        return
    
    rho = lambda_rate / mu_rate
    pi_0 = 1 - rho
    
    n_values = np.arange(0, max_n + 1)
    pi_n_values = pi_0 * (rho ** n_values)
    
    plt.figure(figsize=(12, 6))
    plt.bar(n_values, pi_n_values, alpha=0.7, label=f'$\pi_n$ for $\rho={rho:.2f}$')
    plt.title('Stationary Distribution of an M/M/1 Queue')
    plt.xlabel('Number of Customers in System (n)')
    plt.ylabel('Probability ($\pi_n$)')
    plt.xticks(n_values)
    plt.legend()
    plt.grid(axis='y')
    plt.show()
    
    print(f"Probability system is empty (pi_0): {pi_n_values[0]:.4f}")
    print(f"Probability of 5 customers (pi_5): {pi_n_values[5]:.4f}")
    print(f"Sum of probabilities up to n={max_n}: {np.sum(pi_n_values):.4f}")

# Use the coffee shop example from Week 12
lambda_rate_shop = 20.0
mu_rate_shop = 30.0
plot_mm1_distribution(lambda_rate_shop, mu_rate_shop)

# Example with higher utilization
plot_mm1_distribution(lambda_rate=28.0, mu_rate=30.0)

**Interpretation:**

The plots show the geometric nature of the M/M/1 stationary distribution. 
- When utilization \(\rho\) is low, the probability of having many customers in the system drops off very quickly.
- As utilization gets higher (e.g., \(\rho = 0.93\) in the second plot), the probability of being empty (\(\pi_0\)) becomes very small, and the distribution has a much "heavier tail," meaning there's a significant chance of finding many customers in the system.

## Summary & Course Conclusion

In this final notebook, we've seen how the **Birth-and-Death Process** provides a unifying theoretical framework:
1.  It models any continuous-time Markov process where the state only changes by +1 or -1.
2.  The system is fully described by state-dependent **birth rates (\(\lambda_n\))** and **death rates (\(\mu_n\))**.
3.  The **balance equations** provide a straightforward method for finding the long-run stationary distribution, \(\pi_n\).
4.  We successfully re-derived the M/M/1 queue's stationary distribution, showing it is a specific instance of this more general model.

Congratulations on completing the journey through Probability and Stochastic Processes! You have built a strong foundation, moving from basic definitions to random walks, Markov chains, Poisson processes, queuing theory, and finally to the general birth-and-death framework that ties many of these concepts together. You are now well-equipped to model and analyze a wide variety of random systems in the real world.